In [45]:
%matplotlib inline

import random
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import torch
import dense_correspondence_manipulation.utils.utils as utils
utils.add_dense_correspondence_to_python_path()

import dense_correspondence
from dense_correspondence.evaluation.evaluation import *
import dense_correspondence.correspondence_tools.correspondence_plotter as correspondence_plotter
from dense_correspondence.dataset.dense_correspondence_dataset_masked import ImageType

In [46]:
config_filename = os.path.join(utils.getDenseCorrespondenceSourceDir(), 'config', 
                               'dense_correspondence', 'evaluation', 'evaluation.yaml')
config = utils.getDictFromYamlFilename(config_filename)
default_config = utils.get_defaults_config()


utils.set_cuda_visible_devices([0])
dce = DenseCorrespondenceEvaluation(config)
DCE = DenseCorrespondenceEvaluation


# network_name = "shoes_consistent_M_background_1.000_3"



log_list = []



# log_list = ["2018-05-18-15-57-44"] # many shoes





# mugs
network_name = "rope_rigid_norm_3"
# log_list.append("2018-05-18-16-26-26") # many mugs, moving robot
log_list.append("rope_blender") # may mugs, stationary robot


# shoes
# log_list.append("2018-05-18-16-29-42") # many shoes
 
# hats
# log_list.append("2018-05-18-16-22-08") # many hats









dcn = dce.load_network_from_config(network_name)
dcn.eval()
dataset = dcn.load_training_dataset()

setting CUDA_VISIBLE_DEVICES =  0,
printing config {'logs_root_path': 'code/data_volume/pdc_synthetic_2/logs_proto', 'multi_object': {'test': [], 'train': [], 'evaluation_labeled_data_path': []}, 'single_object': {'rope_rigid': {'logs_root_path': 'code/data_volume/pdc_synthetic_2/logs_proto', 'test': ['rope_rigid_test'], 'train': ['rope_rigid'], 'object_id': 'rope_rigid'}}}
Using SpartanDataset:
   - in train mode
   - number of scenes 1
   - total images:     3730


In [47]:
from PIL import Image
from plotting import normalize_descriptor
import time

## Usage:
## set the network above
## set the full_rgb_only_path in last cell of notebook (if you'd like can use log_list too)
## run the full notebook

descriptor_image_stats = dcn.descriptor_image_stats

def make_descriptors_images(log_folder, save_images_dir, make_masked_video=False):
    image_folder = rgb_filename = os.path.join(log_folder, "processed", "images")
    print(image_folder)
    for img_file in sorted(os.listdir(image_folder)):
        start = time.time()
        if "rgb.png" not in img_file:
            continue
        
        idx_str = img_file.split("_rgb")[0]
        img_file_fullpath = os.path.join(image_folder, img_file)
        rgb_a = Image.open(img_file_fullpath).convert('RGB')

        # compute dense descriptors
        # This takes in a PIL image!
        rgb_a_tensor = dataset.rgb_image_to_tensor(rgb_a)

        # these are Variables holding torch.FloatTensors, first grab the data, then convert to numpy
        res_a = dcn.forward_single_image_tensor(rgb_a_tensor).data.cpu().numpy()
        res_a = normalize_descriptor(res_a, descriptor_image_stats["mask_image"])

        # This chunk of code would produce masked descriptors
        # MASK
        if make_masked_video:
            mask_name = idx_str + "_mask.png"
            mask_filename = os.path.join(log_folder, "processed", "image_masks", mask_name)
            mask = np.asarray(Image.open(mask_filename))
            mask_three_channel = np.zeros((480,640,3))
            for j in range(3):
                mask_three_channel[:,:,j] = mask
            res_a_masked = res_a * mask_three_channel

        
        
        # save rgb image, descriptor image, masked descriptor image
        
        save_file_name = os.path.join(save_images_dir, idx_str + "_res.png")
        plt.imsave(save_file_name, res_a)
        print "forward and saving at rate", time.time()-start

In [48]:
def make_videos(log_folder, save_images_dir, make_masked_video=False):
    # make an rgb only dir
    log_name = os.path.basename(log_folder)
    print "log_name", log_name
    processed_folder = os.path.join(log_folder, 'processed')
    videos_folder = os.path.join(processed_folder, 'videos')
    if not os.path.isdir(videos_folder):
        os.makedirs(videos_folder)
    
    
    
#     rgb_only_path = os.path.join(os.path.dirname(full_rgb_only_path),"rgb_only")
#     os.system("mkdir -p "+ rgb_only_path)
#     os.system("cp "+full_rgb_only_path+"/*rgb.png "+ rgb_only_path)
    
    # make descriptor video
    print "making descriptor video"
    video_des = log_name + "_video_descriptors.mp4"
    video_des_full_filename = os.path.join(videos_folder, video_des)
    os.chdir(save_images_dir)
    cmd = "ffmpeg -framerate 30 -pattern_type glob -i '*res.png' -c:v libx264 -r 30 "\
              + video_des_full_filename
        
    print "descriptor video command:\n", cmd
    os.system(cmd)
    print "done making descriptors"
    
    # make rgb video
    # save it in log_folder/processed/videos
    print "making rgb video"
    os.chdir(save_images_dir)
    video_rgb = log_name + "_video_rgb.mp4"
    video_rgb_full_filename = os.path.join(videos_folder, video_rgb)
    
    rgb_images_folder = os.path.join(processed_folder, 'images')
    print "rgb_images_folder", rgb_images_folder
    os.chdir(rgb_images_folder)
    cmd = "ffmpeg -framerate 30 -pattern_type glob -i '*rgb.png' -c:v libx264 -r 30 " + video_rgb_full_filename
        
    print "rgb video command:\n", cmd
    os.system(cmd)
    
    
    # make rgb video mac friendly
    os.chdir(videos_folder)
    cmd = "ffmpeg \
      -i "+ video_rgb + " -pix_fmt yuv420p " + video_rgb.split(".mp4")[0]+"_mac.mp4"
    print cmd
    os.system(cmd)
    
    # make descriptor video mac friendly
    os.chdir(videos_folder)
    cmd = "ffmpeg \
      -i "+ video_des + " -pix_fmt yuv420p "+ video_des.split(".mp4")[0] + "_mac.mp4"
    os.system(cmd)
    
    
    # merge the videos!
    os.chdir(videos_folder)
    cmd = "ffmpeg \
      -i "+ video_rgb +" \
      -i "+ video_des +"\
      -filter_complex '[0:v]pad=iw*2:ih[int];[int][1:v]overlay=W/2:0[vid]' \
      -map [vid] \
      -c:v libx264 \
      -crf 23 \
      -preset veryfast \
      output_" + log_name + ".mp4"

    print cmd
    os.system(cmd)

In [49]:
# log_list = ["2018-04-10-16-02-59",
#             "2018-04-10-16-14-41",
#             "2018-04-10-16-05-17",
#             "2018-04-10-16-06-26",
#             "2018-04-10-16-12-21",
#             "2018-04-10-16-13-37"]

# log_list = ['2018-04-27-19-39-51'] # caterpillar tricks

# log_list  = ['2018-05-18-16-00-28'] # hats in isolation

# log_list = ['2018-05-18-16-04-03'] # many hats

# stationary robot, moving objects

# log_list = ["2018-05-18-15-57-44"] # many shoes




# moving robot, stationary objects
# log_list += "2018-05-18-16-26-26" # many mugs
# log_list += "2018-05-18-16-29-42" # many shoes
 


logs_special_prefix = "/home/priya/code/data_volume/pdc_synthetic_2/logs_proto"
make_masked_video = False

for log in log_list:
    log_folder = os.path.join(logs_special_prefix, log)
    print(log_folder)
    save_images_dir = os.path.join(log_folder, "processed", "images")
    if not os.path.isdir(save_images_dir):
        os.makedirs(save_images_dir)
    print "save_images_dir", save_images_dir
    
    make_descriptors_images(log_folder, save_images_dir, make_masked_video=make_masked_video)
    make_videos(log_folder, save_images_dir, make_masked_video=make_masked_video)

/home/priya/code/data_volume/pdc_synthetic_2/logs_proto/rope_blender
save_images_dir /home/priya/code/data_volume/pdc_synthetic_2/logs_proto/rope_blender/processed/images
/home/priya/code/data_volume/pdc_synthetic_2/logs_proto/rope_blender/processed/images
normalizing descriptor norm
forward and saving at rate 0.161675214767
normalizing descriptor norm
forward and saving at rate 0.153245925903
normalizing descriptor norm
forward and saving at rate 0.153362035751
normalizing descriptor norm
forward and saving at rate 0.155255079269
normalizing descriptor norm
forward and saving at rate 0.152491092682
normalizing descriptor norm
forward and saving at rate 0.153081893921
normalizing descriptor norm
forward and saving at rate 0.153356075287
normalizing descriptor norm
forward and saving at rate 0.152850866318
normalizing descriptor norm
forward and saving at rate 0.152529001236
normalizing descriptor norm
forward and saving at rate 0.152969121933
normalizing descriptor norm
forward and sav

forward and saving at rate 0.155254840851
normalizing descriptor norm
forward and saving at rate 0.154840946198
normalizing descriptor norm
forward and saving at rate 0.157264947891
normalizing descriptor norm
forward and saving at rate 0.157400846481
normalizing descriptor norm
forward and saving at rate 0.15837597847
normalizing descriptor norm
forward and saving at rate 0.157124042511
normalizing descriptor norm
forward and saving at rate 0.157871961594
normalizing descriptor norm
forward and saving at rate 0.158125162125
normalizing descriptor norm
forward and saving at rate 0.157645940781
normalizing descriptor norm
forward and saving at rate 0.158066987991
normalizing descriptor norm
forward and saving at rate 0.157911062241
normalizing descriptor norm
forward and saving at rate 0.152740955353
normalizing descriptor norm
forward and saving at rate 0.152948141098
normalizing descriptor norm
forward and saving at rate 0.153160810471
normalizing descriptor norm
forward and saving at

forward and saving at rate 0.157387018204
normalizing descriptor norm
forward and saving at rate 0.156841993332
normalizing descriptor norm
forward and saving at rate 0.159224033356
normalizing descriptor norm
forward and saving at rate 0.156529188156
normalizing descriptor norm
forward and saving at rate 0.156640052795
normalizing descriptor norm
forward and saving at rate 0.153146028519
normalizing descriptor norm
forward and saving at rate 0.153065919876
normalizing descriptor norm
forward and saving at rate 0.153327941895
normalizing descriptor norm
forward and saving at rate 0.15362906456
normalizing descriptor norm
forward and saving at rate 0.154153823853
normalizing descriptor norm
forward and saving at rate 0.153359889984
normalizing descriptor norm
forward and saving at rate 0.15328502655
normalizing descriptor norm
forward and saving at rate 0.153273105621
normalizing descriptor norm
forward and saving at rate 0.15410399437
normalizing descriptor norm
forward and saving at r

forward and saving at rate 0.153091907501
normalizing descriptor norm
forward and saving at rate 0.154438972473
normalizing descriptor norm
forward and saving at rate 0.154479026794
normalizing descriptor norm
forward and saving at rate 0.154448986053
normalizing descriptor norm
forward and saving at rate 0.154541015625
normalizing descriptor norm
forward and saving at rate 0.155277013779
normalizing descriptor norm
forward and saving at rate 0.155040025711
normalizing descriptor norm
forward and saving at rate 0.15606713295
normalizing descriptor norm
forward and saving at rate 0.155996084213
normalizing descriptor norm
forward and saving at rate 0.154989004135
normalizing descriptor norm
forward and saving at rate 0.156141042709
normalizing descriptor norm
forward and saving at rate 0.157875061035
normalizing descriptor norm
forward and saving at rate 0.154937028885
normalizing descriptor norm
forward and saving at rate 0.155577898026
normalizing descriptor norm
forward and saving at

forward and saving at rate 0.153913974762
normalizing descriptor norm
forward and saving at rate 0.153737068176
normalizing descriptor norm
forward and saving at rate 0.153970003128
normalizing descriptor norm
forward and saving at rate 0.153699159622
normalizing descriptor norm
forward and saving at rate 0.153396844864
normalizing descriptor norm
forward and saving at rate 0.153455018997
normalizing descriptor norm
forward and saving at rate 0.153995037079
normalizing descriptor norm
forward and saving at rate 0.153161048889
normalizing descriptor norm
forward and saving at rate 0.153611183167
normalizing descriptor norm
forward and saving at rate 0.15314912796
normalizing descriptor norm
forward and saving at rate 0.154484987259
normalizing descriptor norm
forward and saving at rate 0.153539896011
normalizing descriptor norm
forward and saving at rate 0.153776884079
normalizing descriptor norm
forward and saving at rate 0.153466939926
normalizing descriptor norm
forward and saving at

forward and saving at rate 0.158811092377
normalizing descriptor norm
forward and saving at rate 0.15914106369
normalizing descriptor norm
forward and saving at rate 0.159567117691
normalizing descriptor norm
forward and saving at rate 0.158532857895
normalizing descriptor norm
forward and saving at rate 0.158571004868
normalizing descriptor norm
forward and saving at rate 0.160336971283
normalizing descriptor norm
forward and saving at rate 0.158228874207
normalizing descriptor norm
forward and saving at rate 0.159376144409
normalizing descriptor norm
forward and saving at rate 0.158546924591
normalizing descriptor norm
forward and saving at rate 0.157377004623
normalizing descriptor norm
forward and saving at rate 0.157835006714
normalizing descriptor norm
forward and saving at rate 0.158612012863
normalizing descriptor norm
forward and saving at rate 0.15961098671
normalizing descriptor norm
forward and saving at rate 0.159437179565
normalizing descriptor norm
forward and saving at 

forward and saving at rate 0.103923082352
normalizing descriptor norm
forward and saving at rate 0.103660106659
normalizing descriptor norm
forward and saving at rate 0.10409617424
normalizing descriptor norm
forward and saving at rate 0.103395938873
normalizing descriptor norm
forward and saving at rate 0.103502988815
normalizing descriptor norm
forward and saving at rate 0.103594064713
log_name rope_blender
making descriptor video
descriptor video command:
ffmpeg -framerate 30 -pattern_type glob -i '*res.png' -c:v libx264 -r 30 /home/priya/code/data_volume/pdc_synthetic_2/logs_proto/rope_blender/processed/videos/rope_blender_video_descriptors.mp4
done making descriptors
making rgb video
rgb_images_folder /home/priya/code/data_volume/pdc_synthetic_2/logs_proto/rope_blender/processed/images
rgb video command:
ffmpeg -framerate 30 -pattern_type glob -i '*rgb.png' -c:v libx264 -r 30 /home/priya/code/data_volume/pdc_synthetic_2/logs_proto/rope_blender/processed/videos/rope_blender_video_r

In [6]:
# sandboxing
#os.system("cd /media/peteflo/3TBbackup/dense_correspondence/pdc/logs_proto/2018-04-10-16-13-37/processed && ffmpeg -i /media/peteflo/3TBbackup/dense_correspondence/pdc/logs_proto/2018-04-10-16-13-37/processed/rgb_only/2018-04-10-16-13-37_video_rgb.mp4   -c:v libx264       -crf 23       -preset veryfast /media/peteflo/3TBbackup/dense_correspondence/pdc/logs_proto/2018-04-10-16-13-37/processed/rgb_only/2018-04-10-16-13-37_video_rgb_2.mp4")

In [ ]:
# os.system("cd /media/peteflo/3TBbackup/dense_correspondence/pdc/logs_proto/2018-04-10-16-14-41/processed && ffmpeg       -i /media/peteflo/3TBbackup/dense_correspondence/pdc/logs_proto/2018-04-10-16-14-41/processed/rgb_only/2018-04-10-16-14-41_video_rgb.mp4 -pix_fmt yuv420p /media/peteflo/3TBbackup/dense_correspondence/pdc/logs_proto/2018-04-10-16-14-41/processed/rgb_only/2018-04-10-16-14-41_video_rgb_2.mp4")